<a href="https://colab.research.google.com/github/mushhub/my-first-blockchain/blob/main/Stake_SOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
//@version=5
indicator("Long-term SMA Convergence Scanner with Ichimoku", overlay=true)

// Define SMAs
sma25 = ta.sma(close, 25)
sma50 = ta.sma(close, 50)
sma75 = ta.sma(close, 75)
sma100 = ta.sma(close, 100)
sma200 = ta.sma(close, 200)

// Check if price was below 200 SMA for at least 52 weeks (1 year)
lookbackPeriod = 52
belowSma200Count = 0
for i = 1 to lookbackPeriod
    if close[i] < sma200[i]
        belowSma200Count := belowSma200Count + 1

longTermBelowSma200 = belowSma200Count >= lookbackPeriod * 0.9  // 90% of the time

// Check for SMA convergence
// Calculate standard deviation between SMAs as a measure of convergence
smaArray = array.new_float(5)
array.set(smaArray, 0, sma25)
array.set(smaArray, 1, sma50)
array.set(smaArray, 2, sma75)
array.set(smaArray, 3, sma100)
array.set(smaArray, 4, sma200)

pastSmaArray = array.new_float(5)
array.set(pastSmaArray, 0, sma25[12])
array.set(pastSmaArray, 1, sma50[12])
array.set(pastSmaArray, 2, sma75[12])
array.set(pastSmaArray, 3, sma100[12])
array.set(pastSmaArray, 4, sma200[12])

stdDevCurrent = array.stdev(smaArray)
stdDevPast = array.stdev(pastSmaArray)

convergingSmas = stdDevCurrent < stdDevPast * 0.8  // SMAs are at least 20% more converged than 12 weeks ago

// Check for price crossing above 200 SMA
crossedAboveSma200 = ta.crossover(close, sma200)

// Alert condition
alertCondition = longTermBelowSma200 and convergingSmas and crossedAboveSma200

// ----- 一目均衡表（Ichimoku Kinko Hyo）の追加 -----
// パラメータ設定
conversionPeriods = input.int(9, "転換線期間(Conversion Line Period)")
basePeriods = input.int(26, "基準線期間(Base Line Period)")
laggingSpan2Periods = input.int(52, "先行スパン2期間(Lagging Span 2 Period)")
displacement = input.int(26, "先行スパン変位(Displacement)")

// 各ラインの計算
conversionLine = math.avg(ta.highest(high, conversionPeriods), ta.lowest(low, conversionPeriods))
baseLine = math.avg(ta.highest(high, basePeriods), ta.lowest(low, basePeriods))
leadingSpan1 = math.avg(conversionLine, baseLine)
leadingSpan2 = math.avg(ta.highest(high, laggingSpan2Periods), ta.lowest(low, laggingSpan2Periods))
laggingSpan = close

// 雲（Kumo）の色設定
kumoColorBullish = color.new(color.green, 80)
kumoColorBearish = color.new(color.red, 80)

// 雲の描画
fill(plot(leadingSpan1, "", color=color.new(color.blue, 0), offset=displacement),
     plot(leadingSpan2, "", color=color.new(color.blue, 0), offset=displacement),
     color = leadingSpan1 > leadingSpan2 ? kumoColorBullish : kumoColorBearish,
     title="Kumo Cloud")

// 各ラインの描画
// plot(conversionLine, "転換線(Conversion Line)", color=color.blue)
// plot(baseLine, "基準線(Base Line)", color=color.red)
// plot(close, "現在値(Current Price)", color=color.purple, offset=-displacement, title="遅行スパン(Lagging Span)")
// plot(leadingSpan1, "先行スパン1(Leading Span 1)", color=color.green, offset=displacement)
// plot(leadingSpan2, "先行スパン2(Leading Span 2)", color=color.red, offset=displacement)

// 一目均衡表による追加シグナル
ichimokuBullish = close > baseLine and close > leadingSpan1[displacement] and close > leadingSpan2[displacement]
ichimokuConfirmation = ta.crossover(conversionLine, baseLine) and ichimokuBullish

// 元のシグナルと一目均衡表シグナルの組み合わせ
combinedSignal = alertCondition and ichimokuConfirmation

// Plot signals
plotshape(alertCondition, title="SMA Pattern", location=location.belowbar, color=color.green, style=shape.triangleup, size=size.normal)
// plotshape(combinedSignal, title="Combined Pattern", location=location.belowbar, color=color.blue, style=shape.triangleup, size=size.large)

// Plot SMAs
plot(sma25, color=color.new(color.blue, 50), title="25 SMA", linewidth=2)
plot(sma50, color=color.new(color.orange, 50), title="50 SMA", linewidth=2)
plot(sma75, color=color.new(color.red, 50), title="75 SMA", linewidth=2)
plot(sma100, color=color.new(color.green, 50), title="100 SMA", linewidth=2)
plot(sma200, color=color.new(color.black, 50), title="200 SMA", linewidth=2)

// Create alerts
alertcondition(alertCondition, title="SMA Convergence Pattern", message="SMA Convergence Pattern detected on {{ticker}}")
alertcondition(ichimokuConfirmation, title="Ichimoku Bullish Signal", message="Ichimoku Bullish Signal on {{ticker}}")
alertcondition(combinedSignal, title="Combined SMA & Ichimoku Pattern", message="Strong bullish pattern: SMA Convergence with Ichimoku confirmation on {{ticker}}")